# Customer Revenue Prediction

## Exploratory Data Analysis
*Machine Learning Nanodegree Program | Capstone Project*

---

In this notebook I will be exploring the dataset, noting patterns about the data points and distribution of the dataset.

### Overview:
- Reading the data
- Filling in the missing data
- Analyzing the dataset
- Removing the constant value columns
- Normalizing the dataset 
- Spliting the data into training, validation and test sets
- Saving the files

### Dataset:
The dataset is download from the [Kaggle competetion](https://www.kaggle.com/c/ga-customer-revenue-prediction/overview) Google Analytics Customer Revenue Prediction.

#### Initial Data:
- **fullVisitorId**- A unique identifier for each user of the Google Merchandise Store.
- **channelGrouping** - The channel via which the user came to the Store.
- **date** - The date on which the user visited the Store.
- **sessionId** - The sessionId of the user visited the Store.
- **device** - The specifications for the device used to access the Store.
- **geoNetwork** - This section contains information about the geography of the user.
- **socialEngagementType** - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
- **totals** - This section contains aggregate values across the session.
- **trafficSource** - This section contains information about the Traffic Source from which the session originated.
- **visitId** - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
- **visitNumber** - The session number for this user. If this is the first session, then this is set to 1.
- **visitStartTime** - The timestamp (expressed as POSIX time).
---

### Loading and formatting Data

First, import the relevant libraries into notebook

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import squarify
import plotly.graph_objs as go 

from os import path
from datetime import datetime
from dateutil.parser import parse
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from plotly.offline import iplot, plot 
from pandas.io.json import json_normalize

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.10f' % x)

Set the various paths for the files that are used for reading and writing the data

In [2]:
data_dir = '../datasets'

if not path.exists(data_dir):
    raise Exception('{} directory not found.'.format(data_dir))

raw_train_file = '{}/{}'.format(data_dir, 'raw_train.zip')
print('\nRaw Train file: {}'.format(raw_train_file))

raw_test_file = '{}/{}'.format(data_dir, 'raw_test.zip')
print('\nRaw Test file: {}'.format(raw_test_file))

train_file = '{}/{}'.format(data_dir, 'train.zip')
print('\nTrain file: {}'.format(train_file))

val_file = '{}/{}'.format(data_dir, 'val.zip')
print('\nValidation file: {}'.format(val_file))

test_file = '{}/{}'.format(data_dir, 'test.zip')
print('\nTest file: {}'.format(test_file))


Raw Train file: ../datasets/raw_train.zip

Raw Test file: ../datasets/raw_test.zip

Train file: ../datasets/train.zip

Validation file: ../datasets/val.zip

Test file: ../datasets/test.zip


Some of the columns in the intial data are JSON objects. We need to serialize them so as to access the various data points in the JSON object.

In [3]:
def load_data(data_path):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

    df = pd.read_csv(
        data_path,
        converters={column: json.loads for column in JSON_COLUMNS},
        dtype={'fullVisitorId': 'str'},
        compression='zip'
    )

    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f'{column}.{subcolumn}' for subcolumn in column_as_df.columns]

        df.drop(column, axis=1, inplace=True)
        df = df.merge(column_as_df, right_index=True, left_index=True)

    [rows, columns] = df.shape

    print('Loaded {} rows with {} columns from {}.'.format(
        rows, columns, data_path
    ))
    
    return df

In [4]:
%%time 

train_df = load_data(raw_train_file)
test_df = load_data(raw_test_file)
print()

Loaded 903653 rows with 55 columns from ../datasets/raw_train.zip.
Loaded 804684 rows with 53 columns from ../datasets/raw_test.zip.

CPU times: user 4min 20s, sys: 27 s, total: 4min 47s
Wall time: 5min 46s


After the import and transformation, we have 55 columns.
Now, lets see our data and handle them

In [ ]:
train_df.head()

In [5]:
%%time 

total_null_sum = train_df.isnull().sum()
total_null_count = train_df.isnull().count()

train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].astype('float')

dfgv_id = train_df.groupby('fullVisitorId')
dfgv_id = dfgv_id['totals.transactionRevenue'].sum().reset_index()

total = total_null_sum.sort_values(ascending=False)
percent = (total_null_sum / total_null_count * 100)
percent = percent.sort_values(ascending=False)

df = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

rows = train_df.shape[0]
uniq_rows = dfgv_id.shape[0]
uniq_visitors = train_df['fullVisitorId'].nunique()
    
nzi = pd.notnull(train_df["totals.transactionRevenue"]).sum()
nzr = (dfgv_id['totals.transactionRevenue'] > 0).sum()

print('\n{} unique customers in train set. Ratio: {}.'.format(uniq_visitors, (uniq_visitors / rows)))
print('{} instances have non-zero revenue out of {}. Ratio: {}.'.format(nzi, rows, (nzi / rows)))
print('{} unique customers with non-zero revenue out of {}. Ratio: {}.'.format(nzr, uniq_rows, (nzr / uniq_rows)))
print('{} number of common visitors in train and test set.'.format(
    len(set(train_df['fullVisitorId'].unique()).intersection(set(test_df['fullVisitorId'].unique())))
))

print('\nTotal columns with at least one Values:')
print(df[~(df['Total'] == 0)])
print()


714167 unique customers in train set. Ratio: 0.7903111039303804.
11515 instances have non-zero revenue out of 903653. Ratio: 0.0127427231470487.
9996 unique customers with non-zero revenue out of 714167. Ratio: 0.013996726255903731.
7679 number of common visitors in train and test set.

Total columns with at least one Values:
                                               Total       Percent
trafficSource.campaignCode                    903652 99.9998893381
trafficSource.adContent                       892707 98.7886943329
totals.transactionRevenue                     892138 98.7257276853
trafficSource.adwordsClickInfo.page           882193 97.6251946267
trafficSource.adwordsClickInfo.adNetworkType  882193 97.6251946267
trafficSource.adwordsClickInfo.slot           882193 97.6251946267
trafficSource.adwordsClickInfo.isVideoAd      882193 97.6251946267
trafficSource.adwordsClickInfo.gclId          882092 97.6140177701
trafficSource.isTrueDirect                    629648 69.6780733312
t

From the above results we see that some columns have null values. We will be defaulting the values for the columns that have less than 90% null values and remove the columns that have more than 90% null values

In [ ]:
print('\nOriginal Dataframe Info:\n')

print(train_df.info())
print()

In [6]:
%%time

na_val_keys = {
    'totals.pageviews': [1, 'int'],
    'totals.newVisits': [0, 'int'],
    'totals.bounces': [0, 'int'],
    'totals.transactionRevenue': [0, 'float'],
    'trafficSource.isTrueDirect': [False, 'bool'],
    'trafficSource.referralPath': ['N/A', 'str']
}

for nav_k, [nav_v, nav_t] in na_val_keys.items():
    train_df[nav_k] = train_df[nav_k].fillna(nav_v)
    train_df[nav_k] = train_df[nav_k].astype(nav_t)
    
    if nav_k != 'totals.transactionRevenue':
        test_df[nav_k] = test_df[nav_k].fillna(nav_v)
        test_df[nav_k] = test_df[nav_k].astype(nav_t)
    
na_vals = [
    'unknown.unknown', '(not set)', 'not available in demo dataset', 
    '(not provided)', '(none)', '<NA>', 'nan'
]

for na_val in na_vals:
    train_df = train_df.replace(na_val, 'N/A')
    test_df = test_df.replace(na_val, 'N/A')

CPU times: user 1min 24s, sys: 16.2 s, total: 1min 40s
Wall time: 1min 57s


There may be some columns that have unique values which may not be useful in our model training. So we need to remove those columns along with the columns that have 90% of values as null. 

In [7]:
%%time

const_cols = [
    c
    for c in train_df.columns
    if train_df[c].nunique(dropna=False) == 1
]

const_cols += [
    'sessionId', 'visitId', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.slot',
    'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.gclId',
    'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.isVideoAd',
]

cols_not_in_test = set(train_df.columns).difference(set(test_df.columns))

train_const_cols = const_cols + ['trafficSource.campaignCode']

train_df.drop(train_const_cols, axis=1, inplace=True)
test_df.drop(const_cols, axis=1, inplace=True)

print('\nTotal Train Features dropped : {}'.format(train_const_cols))
print('\nTrain features dropped: {}'.format(len(train_const_cols)))

print('\nTotal Test Features dropped : {}'.format(const_cols))
print('\nTest features dropped: {}'.format(len(const_cols)))

print('\nTrain Shape after dropping: {}'.format(train_df.shape))
print('\nTest Shape after dropping: {}\n'.format(test_df.shape))


Total Train Features dropped : ['socialEngagementType', 'device.browserSize', 'device.browserVersion', 'device.flashVersion', 'device.language', 'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel', 'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits', 'trafficSource.adwordsClickInfo.criteriaParameters', 'sessionId', 'visitId', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.adwordsClickInfo.page', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.campaignCode']

Train features dropped: 28

Total Test Features dropped : ['socialEngagementType', 'device.browserSize', 'device.browserVersion', 'device.flashVersion', 'devic

In [ ]:
print(train_df.info())
print()

### Visualizations

Let us visualize some patterns between the various features.
<br />
<div align="right">
    <font size="2">
        Used the visualizations from <a href="https://www.kaggle.com/kabure/exploring-the-consumer-patterns-ml-pipeline">here</a> 
    </font>
</div>


#### Revenue Distribution 

In [ ]:
plt.figure(figsize=(14,6))

plt.subplot(1,2,1)

ax = sns.distplot(
    np.log1p(train_df[train_df['totals.transactionRevenue'] > 0]["totals.transactionRevenue"] + 0.01), 
    bins=40, 
    kde=True
)
ax.set_xlabel('Transaction RevenueLog', fontsize=12) 
ax.set_ylabel('Distribuition', fontsize=12) 
ax.set_title("Distribuition of Revenue Log", fontsize=12) 

plt.subplot(1,2,2)

plt.scatter(range(train_df.shape[0]), np.sort(train_df['totals.transactionRevenue'].values))
plt.xlabel('Index', fontsize=12)
plt.ylabel('Revenue value', fontsize=12) 
plt.title("Revenue Value Distribution", fontsize=12) 

plt.show()

#### Crossing Channel Grouping x Browsers

In [ ]:
crosstab_eda = pd.crosstab(
    index=train_df['channelGrouping'], normalize=True,
    columns=train_df[train_df['device.browser'].isin(train_df['device.browser'].value_counts()[:5].index.values)]['device.browser']
)

crosstab_eda.plot(
    kind="bar",    
    figsize=(14,6),
    stacked=True
)

plt.title("Channel Grouping % for which Browser", fontsize=12) 
plt.xlabel("The Channel Grouping Name", fontsize=12) 
plt.ylabel("Count", fontsize=12) 
plt.xticks(rotation=0)

plt.show() 

#### Crossing Operation Systems x Browsers

In [ ]:
crosstab_eda = pd.crosstab(
    index=train_df[train_df['device.operatingSystem'].isin(train_df['device.operatingSystem'].value_counts()[:6].index.values)]['device.operatingSystem'], 
    columns=train_df[train_df['device.browser'].isin(train_df['device.browser'].value_counts()[:5].index.values)]['device.browser'])

crosstab_eda.plot(
    kind="bar",    
    figsize=(14,6), 
    stacked=True
)   

plt.title("Most frequent OS's by Browsers of users", fontsize=12) 
plt.xlabel("Operational System Name", fontsize=12) 
plt.ylabel("Count OS", fontsize=12) 
plt.xticks(rotation=0) 

plt.show()

#### Crossing Device Catgory x Operating Systems

In [ ]:
crosstab_eda = pd.crosstab(
    index=train_df['device.deviceCategory'], 
    columns=train_df[train_df['device.operatingSystem'].isin(train_df['device.operatingSystem'].value_counts()[:6].index.values)]['device.operatingSystem']
)

crosstab_eda.plot(
    kind="bar",    
    figsize=(14,6), 
    stacked=True
)   
plt.title("Most frequent OS's by Device Categorys of users", fontsize=12) 
plt.xlabel("Device Name", fontsize=12)                
plt.ylabel("Count Device x OS", fontsize=12)          
plt.xticks(rotation=0)                                

plt.show()

In [ ]:
number_of_colors = 20 

color = [
    "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    for i in range(number_of_colors)
]

#### Exploring Countries

In [ ]:
country_tree = train_df["geoNetwork.country"].value_counts() 

print("Description most frequent countrys: ")
print(country_tree[:15])

country_tree = round(
    (train_df["geoNetwork.country"].value_counts()[:30] / len(train_df['geoNetwork.country']) * 100),
    2
)

plt.figure(figsize=(14,6))

g = squarify.plot(
    sizes=country_tree.values, 
    label=country_tree.index, 
    value=country_tree.values,
    alpha=.4, 
    color=color
)
g.set_title("'TOP 30 Countrys - % size of total",fontsize=12)
g.set_axis_off()
plt.show()


#### Exploring Cities

In [ ]:
city_tree = train_df["geoNetwork.city"].value_counts()

print("Description most frequent Citys: " )
print(city_tree[:15])

city_tree = round(
    (train_df["geoNetwork.city"].value_counts()[:30] / len(train_df['geoNetwork.city']) * 100),
    2
)

plt.figure(figsize=(14,6))

g = squarify.plot(
    sizes=city_tree.values, 
    label=city_tree.index, 
    value=city_tree.values,
    alpha=.4, 
    color=color
)

g.set_title("'TOP 30 Citys - % size of total",fontsize=12)
g.set_axis_off()
plt.show()

In [ ]:
def PieChart(df_colum, title, limit=15):
    count_trace = train_df[df_colum].value_counts()[:limit].to_frame().reset_index()
    rev_trace = train_df.groupby(df_colum)["totals.transactionRevenue"].sum().nlargest(10).to_frame().reset_index()

    trace1 = go.Pie(
        labels=count_trace['index'], 
        values=count_trace[df_colum], 
        name= "% Acesses", 
        hole= .5, 
        hoverinfo="label+percent+name", 
        showlegend=True,
        domain= {'x': [0, .48]}, 
        marker=dict(colors=color)
    )

    trace2 = go.Pie(
        labels=rev_trace[df_colum], 
        values=rev_trace['totals.transactionRevenue'], 
        name="% Revenue", 
        hole= .5, 
        hoverinfo="label+percent+name", 
        showlegend=False, 
        domain= {'x': [.52, 1]}
    )

    layout = dict(
        title= title, 
        height=450, 
        font=dict(size=15),
        annotations = [
            dict(x=.25, y=.5, text='Visits', showarrow=False, font=dict(size=20)),
            dict(x=.80, y=.5, text='Revenue', showarrow=False, font=dict(size=20))
        ]
    )

    fig = dict(data=[trace1, trace2], layout=layout)
    iplot(fig)

#### Device Category feature

In [ ]:
PieChart("device.deviceCategory", "Device Category")

#### Device Borwser feature

In [ ]:
PieChart("device.browser", "Device Browser")

#### Operating System feature

In [ ]:
PieChart("device.operatingSystem", "Operating System")

### Normalizing Data

Now let us encode the features an normilize them so that the values are between 0 and 1. I will be using LabelEncoder from sklearn to encode the values. Then use MinMaxScaler to normalize the feature values between 0 and 1.

In [8]:
%%time

str_cols = [
    'channelGrouping', 'device.browser', 'device.deviceCategory', 'device.isMobile', 
    'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 
    'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 
    'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword', 
    'trafficSource.medium', 'trafficSource.source', 'trafficSource.referralPath'
]

num_cols = [
    'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'visitNumber', 'visitStartTime'
]

feature_cols = str_cols + num_cols

train_core_df = train_df.loc[:, ['fullVisitorId', 'date', 'totals.transactionRevenue']]
train_revenue_df = train_df.loc[:, ['totals.transactionRevenue']]

test_core_df = test_df.loc[:, ['fullVisitorId', 'date']]

train_rest_df = train_df.loc[:, feature_cols]
test_rest_df = test_df.loc[:, feature_cols]

print('Encoding str_cols...')

for str_col in str_cols:
    labelEncoder = LabelEncoder()
    
    train_col_list = list(train_rest_df[str_col].astype('str'))
    test_col_list = list(test_rest_df[str_col].astype('str'))
    
    labelEncoder.fit(train_col_list + test_col_list)
    
    train_rest_df[str_col] = labelEncoder.transform(train_col_list)
    test_rest_df[str_col] = labelEncoder.transform(test_col_list)

print('Encoding num_cols...')

for num_col in num_cols:
    train_rest_df[num_col] = train_rest_df[num_col].astype('float')
    test_rest_df[num_col] = test_rest_df[num_col].astype('float')
    
minMaxScaler = MinMaxScaler()

print('Min Max Scalling train_df...')

normalized_train_df = pd.DataFrame(minMaxScaler.fit_transform(train_rest_df.astype('float')))
normalized_train_df.columns = train_rest_df.columns
normalized_train_df.index = train_rest_df.index

print('Min Max Scalling test_df...')

normalized_test_df = pd.DataFrame(minMaxScaler.transform(test_rest_df.astype('float')))
normalized_test_df.columns = test_rest_df.columns
normalized_test_df.index = test_rest_df.index

# print('Min Max Scalling totals.transactionRevenue...')

# normalized_train_df['totals.transactionRevenue'] = minMaxScaler.fit_transform(train_revenue_df)

print()

Encoding str_cols...
Encoding num_cols...
Min Max Scalling train_df...
Min Max Scalling test_df...

CPU times: user 1min 24s, sys: 16.8 s, total: 1min 41s
Wall time: 2min 11s


In [9]:
normalized_train_df.describe()

,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,...,trafficSource.keyword,trafficSource.medium,trafficSource.source,trafficSource.referralPath,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,visitNumber,visitStartTime
count,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,...,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000,903653.0000000000
mean,0.6617524331,0.3641628231,0.1491850301,0.2646181665,0.5434719820,0.5607796764,0.3483301666,0.7177953151,0.6096716273,0.3047276492,...,0.3970101490,0.7230341735,0.5096656923,0.8399462251,0.4986759298,0.0072074902,0.7780198815,0.0060885659,0.0032103986,0.4734772521
std,0.2535415747,0.1362235360,0.2630287940,0.4411299217,0.3738189781,0.1625162933,0.1868481065,0.3053861596,0.1140410518,0.3575007572,...,0.2530395414,0.3484297969,0.3336830116,0.3048508859,0.4999985235,0.0193215171,0.4155780752,0.0150105752,0.0235627779,0.2853081708
min,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
25%,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.2608695652,0.5796645702,0.2000000000,0.4229074890,0.5950413223,0.0076705019,...,0.1133370432,0.8000000000,0.4168336673,0.8147684606,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.2380072835
50%,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.5796645702,0.2000000000,0.9251101322,0.5950413223,0.1005740966,...,0.6171396772,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0020040080,1.0000000000,0.0000000000,0.0000000000,0.4400086761
75%,1.0000000000,0.3879310345,0.5000000000,1.0000000000,0.8695652174,0.5796645702,0.4000000000,0.9603524229,0.5950413223,0.6819600276,...,0.6171396772,1.0000000000,0.8356713427,1.0000000000,1.0000000000,0.0060120240,1.0000000000,0.0064102564,0.0000000000,0.7186033841
max,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,...,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0000000000


In [10]:
cleaned_train_df = pd.concat([train_core_df, normalized_train_df], axis=1)

cleaned_train_df.head()

,fullVisitorId,date,totals.transactionRevenue,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,...,trafficSource.keyword,trafficSource.medium,trafficSource.source,trafficSource.referralPath,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,visitNumber,visitStartTime
0,1131660440785968503,20160902,0.0000000000,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.3951781971,0.4000000000,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0884048864
1,377306020877927890,20160902,0.0000000000,0.5714285714,0.3534482759,0.0000000000,0.0000000000,0.2608695652,0.5796645702,1.0000000000,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0899785187
2,3895546263509774583,20160902,0.0000000000,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.4968553459,0.6000000000,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0895117291
3,4763447161404445595,20160902,0.0000000000,0.5714285714,0.6724137931,0.0000000000,0.0000000000,0.2173913043,0.5796645702,0.4000000000,...,0.2980894083,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0900122290
4,27294437909732085,20160902,0.0000000000,0.5714285714,0.2844827586,0.5000000000,1.0000000000,0.0000000000,0.5796645702,0.6000000000,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0025380711,0.0881587000


In [11]:
normalized_test_df.describe()

,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,...,trafficSource.keyword,trafficSource.medium,trafficSource.source,trafficSource.referralPath,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,visitNumber,visitStartTime
count,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,...,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000,804684.0000000000
mean,0.6184674445,0.3568401406,0.2066382580,0.3697650258,0.5052654750,0.5591804911,0.3360504248,0.7164030209,0.6057021162,0.2857866307,...,0.3534675826,0.6850333796,0.4540103341,0.8286018396,0.5231221200,0.0064972467,0.7510650143,0.0053916796,0.0035890527,1.3694010425
std,0.2379738553,0.1345381986,0.2869682078,0.4827412776,0.3960511108,0.1711322963,0.1891517113,0.3068608249,0.1253251219,0.3507801588,...,0.2615929909,0.3491887380,0.2991894126,0.3277769763,0.4994653918,0.0164268186,0.4323963355,0.0123624104,0.0239384192,0.2111163112
min,0.0000000000,-0.0086206897,0.0000000000,0.0000000000,0.0000000000,0.0010482180,0.0000000000,0.0000000000,0.0082644628,0.0000238214,...,0.0012984604,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000003162
25%,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.2173913043,0.5796645702,0.2000000000,0.4096916300,0.5950413223,0.0076705019,...,0.1133370432,0.4000000000,0.4168336673,0.8454317897,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,1.1862967956
50%,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.2608695652,0.5796645702,0.2000000000,0.9515418502,0.5950413223,0.0660806594,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,1.3626506176
75%,0.8571428571,0.3534482759,0.5000000000,1.0000000000,0.8695652174,0.5796645702,0.4000000000,0.9603524229,0.5950413223,0.6408918745,...,0.6171396772,1.0000000000,0.4509018036,1.0000000000,1.0000000000,0.0060120240,1.0000000000,0.0064102564,0.0000000000,1.5556176573
max,1.0000000000,1.0948275862,1.0000000000,1.0000000000,1.0000000000,0.9989517820,1.0000000000,0.9955947137,1.0000000000,1.0000238214,...,1.0001854943,1.0000000000,0.9979959920,1.0000000000,1.0000000000,1.0000000000,1.0000000000,1.0662393162,1.1573604061,1.7431643114


In [12]:
cleaned_test_df = pd.concat([test_core_df, normalized_test_df], axis=1)

cleaned_test_df.head()

,fullVisitorId,date,channelGrouping,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,...,trafficSource.keyword,trafficSource.medium,trafficSource.source,trafficSource.referralPath,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,visitNumber,visitStartTime
0,6167871330617112363,20171016,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.2608695652,0.5796645702,0.4000000000,0.7753303965,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0060120240,0.0000000000,0.0064102564,0.0025380711,1.2053555360
1,0643697640977915618,20171016,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.8695652174,0.9937106918,0.6000000000,0.8149779736,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0080160321,1.0000000000,0.0085470085,0.0000000000,1.2061302405
2,6059383810968229466,20171016,0.5714285714,0.2844827586,0.0000000000,0.0000000000,0.2608695652,0.5796645702,0.6000000000,0.2995594714,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0120240481,1.0000000000,0.0128205128,0.0000000000,1.2051087488
3,2376720078563423631,20171016,0.5714285714,0.6120689655,0.5000000000,1.0000000000,1.0000000000,0.5649895178,0.2000000000,0.9603524229,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0140280561,1.0000000000,0.0064102564,0.0000000000,1.2066997106
4,2314544520795440038,20171016,0.5714285714,0.6120689655,0.0000000000,0.0000000000,0.2608695652,0.7872117400,0.2000000000,0.9603524229,...,0.1133370432,0.8000000000,0.4168336673,1.0000000000,0.0000000000,0.0160320641,1.0000000000,0.0064102564,0.0000000000,1.2074558839


### Splitting Data

Now that we have normalized the data we need to split the dataset into development, validation. I will be splitting the data into 
- dev: 85% i.e, date ranges between 20160801 - 20170531
- val: 15% i.e. date ranges between 20170601 - 20170801

In [ ]:
print('Transactions Minimum Date: {}'.format(cleaned_train_df['date'].min()))
print('Transactions Maximum Date: {}\n'.format(cleaned_train_df['date'].max()))

In [16]:
%%time

split_date = pd.datetime(2017, 6, 1)

cleaned_train_df["date"] = cleaned_train_df["date"].astype('str').apply(lambda x: parse(x, yearfirst=True))

dev_df = cleaned_train_df[cleaned_train_df['date'] < split_date]
val_df = cleaned_train_df[cleaned_train_df['date'] >= split_date]

feature_cols = ['fullVisitorId'] + str_cols + num_cols

dev_X = dev_df[['totals.transactionRevenue'] + feature_cols]
val_X = val_df[['totals.transactionRevenue'] + feature_cols]
test_X = cleaned_test_df[feature_cols]

print('\nNumber of instances in train: {} with {} columns.'.format(dev_X.shape[0], dev_X.shape[1]))
print('\nNumber of instances in val: {} with {} columns.'.format(val_X.shape[0], val_X.shape[1]))
print('\nNumber of instances in test: {} with {} columns.'.format(test_X.shape[0], test_X.shape[1]))
print()


Number of instances in train: 765707 with 26 columns.

Number of instances in val: 137946 with 26 columns.

Number of instances in test: 804684 with 25 columns.

CPU times: user 37.4 s, sys: 809 ms, total: 38.2 s
Wall time: 40.4 s


We now write these files in csv format so that we can use them in future

In [17]:
%%time

dev_X.to_csv(train_file, index=False, compression='zip')
val_X.to_csv(val_file, index=False, compression='zip')
test_X.to_csv(test_file, index=False, compression='zip')

CPU times: user 1min 7s, sys: 1.67 s, total: 1min 8s
Wall time: 1min 15s
